In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler,RobustScaler,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Bidirectional, LSTM, SimpleRNN,GRU,Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [12]:
# Load the dataset
df = pd.read_csv('..//..//dataset//preliminary_dataset.csv')  


# Separating features and target
X = df[['Number of Lanes', 'Status of driver', 'Nature of environment', 'Velocity of vehicle', 'Distance between vehicles', 'Breaking capability']]
y = df['Collussion']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
# X_scaled = X

# Reshape input to be 3D [samples, timesteps, features] for LSTM
X_scaled = np.reshape(X_scaled, (X_scaled.shape[0], 1, X_scaled.shape[1]))

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [13]:
print(X_train.shape, y_train.shape,X_scaled.shape)

(80000, 1, 6) (80000,) (100000, 1, 6)


# Model

In [14]:
def build_drnn():
    model = Sequential()
    
    # Input layer
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))
    
    # DRNN layers with 3 hidden layers
    model.add(SimpleRNN(units=100,activation='relu',  return_sequences=True))
    model.add(SimpleRNN(units=100,activation='relu',  return_sequences=True))
    model.add(SimpleRNN(units=100, ))
    
    # Output layer
    model.add(Dense(units=1, activation='sigmoid'))
    
    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['mse', 'mae'])
    
    return model


In [15]:
# Instantiate and train the model
model = build_drnn()
import time
batch_size = X_train.shape[0]  # Use full dataset per batch to match experimental setup
start_time=time.time()
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=batch_size,
    steps_per_epoch=1,
    verbose=1
)
end_time = time.time()

Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step - loss: 0.7205 - mae: 0.5118 - mse: 0.2637
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 717ms/step - loss: 0.6876 - mae: 0.4961 - mse: 0.2473
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 745ms/step - loss: 0.6569 - mae: 0.4805 - mse: 0.2320
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 755ms/step - loss: 0.6281 - mae: 0.4650 - mse: 0.2178
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 783ms/step - loss: 0.6008 - mae: 0.4496 - mse: 0.2044
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 759ms/step - loss: 0.5745 - mae: 0.4342 - mse: 0.1918
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 747ms/step - loss: 0.5484 - mae: 0.4185 - mse: 0.1794
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 758ms/step - loss: 0.5225 - mae: 0.4024 - mse: 0.1674
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 744ms/step - loss: 0.4967 - mae: 0.3860 - mse: 0.1557
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 727ms/step - loss: 0.4707 - mae: 0.3691 - mse: 0.1442
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 741ms/step - loss: 0.4447 - ma

In [16]:
# Calculate total training time
total_training_time = end_time - start_time  # in seconds
epochs = 50 # e.g., 50
steps_per_epoch = 1# e.g., 2000

# Total steps
total_steps = epochs * steps_per_epoch

# Calculate time per step
time_per_step = total_training_time / total_steps  # in seconds
time_per_step_ms = time_per_step * 1000  # Convert to milliseconds
#micro seconds
time_per_step_us = time_per_step_ms * 1000

# Calculate time per epoch
time_per_epoch = total_training_time / epochs

# Display results
print(f"Total Steps: {total_steps}")
print(f"Time per Step: {time_per_step:.6f} sec ({time_per_step_ms:.2f} ms)")
print(f"Time per Step: {time_per_step_us:.2f} µs")
print(f"Time per Epoch: {time_per_epoch:.2f} sec")


Total Steps: 50
Time per Step: 0.964469 sec (964.47 ms)
Time per Step: 964468.69 µs
Time per Epoch: 0.96 sec


In [17]:
from sklearn.metrics import mean_squared_error
import numpy as np
# Make predictions
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error for y_pred (MSE): {mse:.16f}')

# Calculate RMSE
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error (RMSE): {rmse:.16f}')


#ACCUARCY
y_pred = (y_pred > 0.5).astype(int)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.16f}')


625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step
Mean Squared Error for y_pred (MSE): 0.0002525599848013
Root Mean Squared Error (RMSE): 0.0158921359420733
Accuracy: 1.0000000000000000


In [18]:
# import matplotlib.pyplot as plt

# # Visualizing the loss, MSE, and MAE
# plt.figure(figsize=(15, 5))

# # Loss plot
# plt.subplot(1, 3, 1)
# plt.plot(history.history['loss'], label='Train Loss')
# plt.plot(history.history['val_loss'], label='Validation Loss')
# plt.title('Model Loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(loc='upper right')

# # MSE plot
# plt.subplot(1, 3, 2)
# plt.plot(history.history['mse'], label='Train MSE')
# plt.plot(history.history['val_mse'], label='Validation MSE')
# plt.title('Mean Squared Error')
# plt.ylabel('MSE')
# plt.xlabel('Epoch')
# plt.legend(loc='upper right')

# # MAE plot
# plt.subplot(1, 3, 3)
# plt.plot(history.history['mae'], label='Train MAE')
# plt.plot(history.history['val_mae'], label='Validation MAE')
# plt.title('Mean Absolute Error')
# plt.ylabel('MAE')
# plt.xlabel('Epoch')
# plt.legend(loc='upper right')

# plt.tight_layout()
# plt.show()


In [19]:
import time

# Start time
start_time = time.time()

# Make predictions
y_pred = model.predict(X_test)

# End time
end_time = time.time()

# Total AES time
total_time = end_time - start_time  

# AES per sample
aes_per_sample = (total_time / len(X_test)) * 1e6  # Convert to microseconds (µs)

print(f'Total AES Time: {total_time:.6f} seconds')
print(f'AES Time per Sample: {aes_per_sample:.2f} µs/sample')


625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
Total AES Time: 2.824973 seconds
AES Time per Sample: 141.25 µs/sample
